# LangChain Agents: Building Intelligent Tool-Using Systems

## Introduction

**Agents** are LLM-powered systems that can decide which tools to use and in what order to accomplish tasks. They enable dynamic, multi-step reasoning and action.

### What Are Agents?

Agents allow LLMs to:
- **Use tools dynamically**: Search, calculate, query databases
- **Make decisions**: Choose which tool to use based on context
- **Multi-step reasoning**: Chain multiple tool calls together
- **Handle complex tasks**: Break down problems into sub-tasks
- **Adapt to results**: Use tool output to inform next steps

### Agent vs Chain

| Chain | Agent |
|-------|-------|
| Predefined sequence | Dynamic decision-making |
| Fixed steps | Adaptive steps |
| No tool selection | Chooses tools |
| Deterministic | Non-deterministic |

### When to Use Agents?

| ✅ Use Agents For | ❌ Don't Use For |
|-------------------|------------------|
| Dynamic tool selection | Simple fixed workflows |
| Multi-step reasoning | Single LLM call |
| Unknown number of steps | Known sequence |
| Complex problem solving | Simple Q&A |

### ReAct Pattern

Most agents use **ReAct** (Reasoning + Acting):

1. **Thought**: Reason about what to do
2. **Action**: Choose and execute a tool
3. **Observation**: See the result
4. **Repeat**: Until task is complete

---

## Installation & Setup

In [1]:
# Install required packages
# !pip install langchain langchain-openai langchain-community
# !pip install duckduckgo-search wikipedia

import os
from getpass import getpass

# Set API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key: ")

print("API key configured!")

API key configured!


---

## Example 1: Simple Custom Tool

Create a basic tool with the `@tool` decorator:

In [2]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together."""
    return a * b

# Inspect the tool
print(f"Name: {multiply.name}")
print(f"Description: {multiply.description}")
print(f"Args: {multiply.args}")

# Use the tool directly
result = multiply.invoke({"a": 5, "b": 7})
print(f"\nResult: {result}")

Name: multiply
Description: Multiply two numbers together.
Args: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}

Result: 35


---

## Example 2: Multiple Custom Tools

Create a toolkit of related tools:

In [ ]:
from langchain_core.tools import tool
from typing import List

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtract b from a."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide a by b."""
    if b == 0:
        return "Error: Division by zero"
    return a / b

# Create toolkit
math_tools = [add, subtract, multiply, divide]

print("Math toolkit:")
for tool in math_tools:
    print(f"- {tool.name}: {tool.description}")

---

## Example 3: Creating an Agent

Build an agent that can use tools:

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Define tools
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

# Get prompt template
prompt = hub.pull("hwchase17/openai-functions-agent")

# Create LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run agent
result = agent_executor.invoke({"input": "How many letters are in the word 'intelligence'?"})
print(f"\nFinal answer: {result['output']}")

### What Just Happened?

The agent:
1. **Reasoned**: Understood it needed to find word length
2. **Selected tool**: Chose `get_word_length`
3. **Called tool**: With argument "intelligence"
4. **Returned result**: 12 letters

---

## Example 4: Agent with Calculator Tools

Multi-step mathematical reasoning:

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Math toolkit
@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def power(base: float, exponent: float) -> float:
    """Raise base to the power of exponent."""
    return base ** exponent

tools = [add, multiply, power]

# Create agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Complex calculation requiring multiple steps
result = agent_executor.invoke({
    "input": "What is (5 + 3) multiplied by 2, and then raise that to the power of 2?"
})

print(f"\nFinal answer: {result['output']}")

---

## Example 5: Built-in Tools - Web Search

Use DuckDuckGo for web searches:

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

# Create search tool
search = DuckDuckGoSearchRun()

tools = [search]

# Create agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Search the web
result = agent_executor.invoke({
    "input": "What is the current version of Python?"
})

print(f"\nAnswer: {result['output']}")

---

## Example 6: Built-in Tools - Wikipedia

Query Wikipedia for information:

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Create Wikipedia tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

tools = [wikipedia]

# Create agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Query Wikipedia
result = agent_executor.invoke({
    "input": "Who was Alan Turing and what did he contribute to computer science?"
})

print(f"\nAnswer: {result['output']}")

---

## Example 7: Combining Multiple Tools

Agent can choose from multiple tools:

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime

# Custom tools
@tool
def get_current_time() -> str:
    """Get the current time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def calculate_age(birth_year: int) -> int:
    """Calculate age from birth year."""
    current_year = datetime.now().year
    return current_year - birth_year

# Built-in tool
search = DuckDuckGoSearchRun()

# Combine tools
tools = [get_current_time, calculate_age, search]

# Create agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test different tools
questions = [
    "What time is it right now?",
    "How old would someone born in 1990 be?",
    "Search for the latest Python release"
]

for question in questions:
    print(f"\n{'='*80}")
    print(f"Question: {question}")
    result = agent_executor.invoke({"input": question})
    print(f"Answer: {result['output']}")

---

## Example 8: Tool with Complex Input Schema

Define tools with Pydantic for validation:

In [ ]:
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# Define input schema
class StudentInfo(BaseModel):
    """Input for student grade calculation."""
    name: str = Field(description="Student's name")
    grades: List[float] = Field(description="List of grades")
    weights: List[float] = Field(default=None, description="Optional weights for grades")

@tool(args_schema=StudentInfo)
def calculate_student_average(name: str, grades: List[float], weights: List[float] = None) -> str:
    """Calculate a student's weighted or unweighted grade average."""
    if weights:
        if len(grades) != len(weights):
            return "Error: grades and weights must have same length"
        avg = sum(g * w for g, w in zip(grades, weights)) / sum(weights)
    else:
        avg = sum(grades) / len(grades)
    
    return f"{name}'s average: {avg:.2f}"

# Test the tool
result = calculate_student_average.invoke({
    "name": "Alice",
    "grades": [85, 90, 88],
    "weights": [1, 2, 1]
})
print(result)

---

## Example 9: Agent with Memory

Create an agent that remembers conversation:

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create tool
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

# Create agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Add memory
message_history = InMemoryChatMessageHistory()

agent_with_memory = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

# Conversation with memory
config = {"configurable": {"session_id": "test-session"}}

print("First question:")
result = agent_with_memory.invoke(
    {"input": "What is 5 times 7?"},
    config=config
)
print(f"Answer: {result['output']}\n")

print("Follow-up (uses memory):")
result = agent_with_memory.invoke(
    {"input": "What was my previous question?"},
    config=config
)
print(f"Answer: {result['output']}")

---

## Example 10: ReAct Agent (Explicit Reasoning)

See the agent's reasoning process:

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

@tool
def get_temperature(city: str) -> str:
    """Get the temperature for a city (mock data)."""
    temps = {
        "new york": "15°C",
        "london": "12°C",
        "tokyo": "18°C"
    }
    return temps.get(city.lower(), "Temperature data not available")

@tool
def compare_numbers(a: float, b: float) -> str:
    """Compare two numbers and return which is larger."""
    if a > b:
        return f"{a} is greater than {b}"
    elif b > a:
        return f"{b} is greater than {a}"
    else:
        return f"{a} and {b} are equal"

tools = [get_temperature, compare_numbers]

# Create ReAct agent
prompt = hub.pull("hwchase17/react")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# Run agent (see Thought, Action, Observation pattern)
result = agent_executor.invoke({
    "input": "Is it warmer in New York or Tokyo?"
})

print(f"\nFinal answer: {result['output']}")

---

## Agent Types Comparison

### OpenAI Functions Agent
- **Best for**: Modern applications (recommended)
- **Pros**: Reliable, fast, good tool selection
- **Cons**: Requires OpenAI models with function calling

### ReAct Agent
- **Best for**: Explicit reasoning, debugging
- **Pros**: Shows thinking process, works with any LLM
- **Cons**: Slower, more tokens

### Structured Chat Agent
- **Best for**: Complex tool inputs
- **Pros**: Handles structured data well
- **Cons**: More verbose

### Conversational Agent
- **Best for**: Chat applications
- **Pros**: Built-in memory
- **Cons**: Legacy, prefer OpenAI Functions

---

## Best Practices

### ✅ Do

1. **Write clear tool descriptions** (agent uses these to decide)
2. **Keep tools focused** (one tool = one task)
3. **Validate inputs** (use Pydantic schemas)
4. **Set max_iterations** (prevent infinite loops)
5. **Use verbose=True** during development (see reasoning)
6. **Handle errors** (tools can fail)
7. **Test tools independently** before giving to agent

### ❌ Don't

1. **Don't make tools too complex** (agent can't use effectively)
2. **Don't skip descriptions** (agent won't know when to use)
3. **Don't use agents for simple tasks** (chains are better)
4. **Don't give too many tools** (agent gets confused, <10 recommended)
5. **Don't forget rate limits** (tools may call external APIs)
6. **Don't trust blindly** (validate agent outputs)

---

## Common Pitfalls

### ❌ Mistake 1: Vague Tool Descriptions

```python
# Bad - agent won't know when to use
@tool
def process(data):
    """Process data."""
    pass
```

**Solution**: Be specific:
```python
@tool
def calculate_average(numbers: List[float]) -> float:
    """Calculate the arithmetic mean of a list of numbers."""
    return sum(numbers) / len(numbers)
```

### ❌ Mistake 2: No Max Iterations

```python
# Bad - can run forever
agent_executor = AgentExecutor(agent=agent, tools=tools)
```

**Solution**: Set limits:
```python
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=5,
    max_execution_time=60  # seconds
)
```

### ❌ Mistake 3: Too Many Tools

```python
# Bad - agent gets confused with 20+ tools
tools = [tool1, tool2, ..., tool25]
```

**Solution**: Group related tools or use tool selection:
```python
# Keep to <10 tools, or create specialized agents
math_tools = [add, subtract, multiply, divide]
```

---

## Practice Exercises

In [ ]:
# Exercise 1: Create a file management agent
# Tools: create_file, read_file, delete_file
# Test: "Create a file called test.txt with content 'Hello World'"

from langchain_core.tools import tool
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI

# Your code here:
@tool
def create_file(filename: str, content: str) -> str:
    """Create a file with given content."""
    # Implement (use a dict to simulate filesystem)
    pass

# Add read_file and delete_file tools
# Create and test agent

In [ ]:
# Exercise 2: Build a research agent
# Combine Wikipedia + Web Search tools
# Test: "Research the history of Python programming language"

from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper

# Your code here:
# Create tools, agent, test

In [ ]:
# Exercise 3: Create a data analysis agent
# Tools: calculate_mean, calculate_median, find_max, find_min
# Test: "What's the average and maximum of [10, 20, 30, 40, 50]?"

from langchain_core.tools import tool
from typing import List

# Your code here:
@tool
def calculate_mean(numbers: List[float]) -> float:
    """Calculate the arithmetic mean of a list of numbers."""
    return sum(numbers) / len(numbers)

# Add other statistical tools
# Create and test agent

---

## Key Takeaways

### ✅ What We Learned

1. **Agents**: LLMs that decide which tools to use dynamically
2. **ReAct Pattern**: Thought → Action → Observation loop
3. **@tool Decorator**: Create custom tools easily
4. **Built-in Tools**: DuckDuckGo, Wikipedia, and more
5. **Agent Types**: OpenAI Functions (best), ReAct, Structured Chat
6. **AgentExecutor**: Runs agents with safety limits
7. **Tool Schemas**: Pydantic for complex inputs
8. **Memory**: Agents can remember conversation context
9. **Best Practices**: Clear descriptions, focused tools, max iterations

### ⚠️ Important Warnings

1. **Agents are non-deterministic** - same input may produce different outputs
2. **Can be expensive** - multiple LLM calls per task
3. **May fail** - agent might not complete task correctly
4. **Validate outputs** - always check agent results
5. **Set limits** - max_iterations, max_execution_time
6. **Not for production-critical** paths without extensive testing

### 📚 Next Steps

- **langchain_memory.ipynb**: Advanced memory patterns
- **LangGraph**: For complex multi-agent systems
- Production agents: Monitoring, fallbacks, human-in-the-loop

---

## Resources

- [Agents Documentation](https://python.langchain.com/docs/modules/agents/)
- [Tools Documentation](https://python.langchain.com/docs/modules/agents/tools/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [Agent Types](https://python.langchain.com/docs/modules/agents/agent_types/)
- [Custom Tools Guide](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

---

**Next Notebook**: `langchain_memory.ipynb` - Conversation memory patterns and strategies